Metric for importance assesment
-Zonation 2
-Zonation 3
-Zonation 6

Source data variable description
https://www.metsakeskus.fi/sites/default/files/document/mvjaete-koodisto-ja-tietokantakuvaus_0.xlsx


The Finnish Forest Center forest stand classes
Luokat Suomen Metsäkeskuksen metsävarakuvioista
fertilityclass	1	Lehto, letto ja lehtomainen suo (ja ruohoturvekangas)
fertilityclass	2	Lehtomainen kangas, vastaava suo ja ruohoturvekangas
fertilityclass	3	Tuore kangas, vastaava suo ja mustikkaturvekangas
fertilityclass	4	Kuivahko kangas, vastaava suo ja puolukkaturvekangas
fertilityclass	5	Kuiva kangas, vastaava suo ja varputurvekangas
fertilityclass	6	Karukkokangas, vastaava suo (ja jäkäläturvekangas)
fertilityclass	7	Kalliomaa ja hietikko
fertilityclass	8	Lakimetsä ja tunturi

treestand
697	type	1	Inventointi-tyyppi
697	type	2	Laskenta-tyyppi.
697	type	3	Ennuste-tyyppi
(type 3 jätetty pois ja otettu korkein arvo luokasta 1 ja 2)

join standid to treestandsummary by treestandsummaryid
join meanage from treestandsummary by (added) standid

Ageclasses
-40 young (age_class = 1)
-40-140 middle (age_class = 2)
-over 140 old (age_class = 3)

Region of interest
-3 maakuntaa. Pirkanmaa, Kainuu ja Uusimaa
downloaded 12.2.2024 from https://avoin.metsakeskus.fi/aineistot/Metsavarakuviot/Maakunta/

Thresholds
-Zonation-arvo > zonation.percentile(0.7) best 30 %
-Zonation-arvo > zonation.percentile(0.9) best 10 %

# Zonation1

## Protection distribution

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Pirkanmaa.gpkg"

#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YoutPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 767056
treestand 1498783
treestandsummary 1526105
specialfeature 1526105
number of empty ages 7551
767056
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                 2
fertilityclass           2
soiltype                 2
drainagestate          149
ditchingyear        717762
thinningyear        767056
developmentclass     19920
standquality        767056
maintreespecies       7005
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           31
Zonation_median         31
Zonation_min            31
Zonation_max            31
meanage               7551
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.9167438997241067
Zonation raja-arvo top 30%:lle on 0.8190546898840486
Suojellun osuus parhaasta 10 %:sta on 16.360545891001596 %
Suojellun osuus parhaasta 30 %:sta on 10.944756412397437 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,1107.036,4420.852,20.026383,1256.756,5201.464,19.459789
2.0,4527.072,35337.613,11.356096,8425.897,82556.200,9.261049
3.0,6251.514,27883.000,18.314349,13269.392,111631.864,10.623906
4.0,1189.467,10255.750,10.392699,3565.696,44596.614,7.403499
5.0,512.669,5084.466,9.159490,1600.097,14232.614,10.106273
6.0,61.664,374.358,14.142406,168.153,1039.032,13.929348
7.0,65.443,472.857,12.157347,209.025,1095.343,16.025002


## General

In [2]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df


,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,70.0,03,2.0,0.763606,0.913797,0.902689,0.774620,0.955791,61.137875,0.168431,9921,7575.737
2.0,1,1.0,20.0,03,2.0,1.030675,0.751503,0.739852,0.593358,0.851603,49.491985,0.074536,174037,179375.662
3.0,1,1.0,10.0,03,2.0,1.238783,0.670370,0.663981,0.555188,0.779385,51.528034,0.054053,362567,449141.686
4.0,1,1.0,10.0,03,1.0,1.313429,0.646647,0.639550,0.544126,0.751594,55.335098,0.056115,168120,220813.625
5.0,1,1.0,62.0,03,1.0,1.301018,0.685725,0.675872,0.573519,0.781121,58.560366,0.094915,42438,55212.614
6.0,2,3.0,62.0,03,1.0,1.656233,0.617285,0.612519,0.514522,0.738003,67.006357,0.170148,6224,10308.394
7.0,2,1.0,50.0,03,1.0,0.860423,0.739226,0.729109,0.633273,0.822340,105.232295,0.073145,3746,3223.144
8.0,2,1.0,50.0,NaN,1.0,0.523000,0.893223,0.883852,0.816730,0.947197,121.000000,0.000000,1,0.523


In [3]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,A0,2.0,1.026282,0.683527,0.676315,0.560827,0.789307,NaN,0.072970,7551,7749.459
1,1,1.0,3.0,10.0,02,2.0,1.312402,0.634030,0.628115,0.518140,0.755088,20.680199,0.047276,229630,301366.932
2,1,1.0,3.0,10.0,03,2.0,1.163921,0.710679,0.701768,0.585750,0.807941,66.359880,0.070735,527954,614496.506
3,2,1.0,7.0,50.0,04,1.0,1.062376,0.717569,0.708969,0.611001,0.806245,160.327954,0.147840,1921,2040.825


In [4]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,2.0,1.191843,0.684887,0.676759,0.564116,0.789063,52.326447,718020,855767.212
1,1,1.0,3.0,10.0,03,2.0,1.425208,0.725490,0.719422,0.583075,0.833932,59.535898,49036,69886.510


## Saving

In [ ]:
# Not protected
out_file = r"YourPath\Pirkanmaa_Top10_Zonation1_NotProtected.parquet"
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
print(f"Top 10% polygon count is {len(stand_age[condition])}")
print(f"Zonation top 10% threshold is {stand_age[condition].Zonation_mean.quantile(0.9)}")
print(f"Zonation top 30% threshold is {stand_age[condition].Zonation_mean.quantile(0.7)}")
stand_age[condition].to_parquet(out_file)
print(f"saving to {out_file}")

out_file = r"YourPath\Pirkanmaa_Top30_Zonation1_NotProtected.parquet"
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
print(f"Top 30% polygon count is {len(stand_age[condition])}")
print(f"Zonation top 10% threshold is {stand_age[condition].Zonation_mean.quantile(0.9)}")
print(f"Zonation top 30% threshold is {stand_age[condition].Zonation_mean.quantile(0.7)}")
stand_age[condition].to_parquet(out_file)
print(f"saving to {out_file}")

# Protected
out_file = r"YourPath\Pirkanmaa_Top10_Zonation1.parquet"
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9))
print(f"Top 10% polygon count is {len(stand_age[condition])}")
print(f"Zonation top 10% threshold is {stand_age[condition].Zonation_mean.quantile(0.9)}")
print(f"Zonation top 30% threshold is {stand_age[condition].Zonation_mean.quantile(0.7)}")
stand_age[condition].to_parquet(out_file)
print(f"saving to {out_file}")

out_file = r"YourPath\Pirkanmaa_Top30_Zonation1.parquet"
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7))
print(f"Top 30% polygon count is {len(stand_age[condition])}")
print(f"Zonation top 10% threshold is {stand_age[condition].Zonation_mean.quantile(0.9)}")
print(f"Zonation top 30% threshold is {stand_age[condition].Zonation_mean.quantile(0.7)}")
stand_age[condition].to_parquet(out_file)
print(f"saving to {out_file}")

# Zonation2

## Protection distribution

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Pirkanmaa.gpkg"

#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YourPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 767056
treestand 1498783
treestandsummary 1526105
specialfeature 1526105
number of empty ages 7551
767056
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                 2
fertilityclass           2
soiltype                 2
drainagestate          149
ditchingyear        717762
thinningyear        767056
developmentclass     19920
standquality        767056
maintreespecies       7005
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           31
Zonation_median         31
Zonation_min            31
Zonation_max            31
meanage               7551
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.9109607683169176
Zonation raja-arvo top 30%:lle on 0.7893264850769294
Suojellun osuus parhaasta 10 %:sta on 21.397650994905398 %
Suojellun osuus parhaasta 30 %:sta on 13.291418631534382 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,1089.459,4290.233,20.251327,1255.402,5110.809,19.719767
2.0,4785.714,34374.238,12.220939,8279.584,77461.085,9.656542
3.0,8091.664,24761.715,24.629625,15911.264,106615.310,12.985970
4.0,1519.818,8197.627,15.640099,4633.053,39670.376,10.457549
5.0,761.355,4146.693,15.512379,1877.463,12547.921,13.014995
6.0,91.491,403.844,18.470530,252.956,1208.080,17.313468
7.0,81.908,569.630,12.571485,212.057,1316.581,13.872284


## General

In [2]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df


,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,70.0,03,2.0,0.763606,0.895079,0.883526,0.748081,0.945345,61.137875,0.168431,9921,7575.737
2.0,1,1.0,20.0,03,2.0,1.030675,0.709502,0.698309,0.544788,0.825458,49.491985,0.074536,174037,179375.662
3.0,1,1.0,10.0,03,2.0,1.238783,0.627261,0.620585,0.498860,0.752507,51.528034,0.054053,362567,449141.686
4.0,1,1.0,10.0,03,1.0,1.313429,0.602565,0.595004,0.488341,0.719934,55.335098,0.056115,168120,220813.625
5.0,1,1.0,62.0,03,1.0,1.301018,0.640481,0.629414,0.515955,0.747486,58.560366,0.094915,42438,55212.614
6.0,2,3.0,62.0,03,1.0,1.656233,0.606727,0.600654,0.488144,0.733563,67.006357,0.170148,6224,10308.394
7.0,2,1.0,50.0,03,1.0,0.860423,0.726886,0.713597,0.595301,0.822003,105.232295,0.073145,3746,3223.144
8.0,2,1.0,50.0,NaN,1.0,0.523000,0.740531,0.712611,0.593270,0.872276,121.000000,0.000000,1,0.523


In [3]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,A0,2.0,1.026282,0.623958,0.618719,0.494880,0.746163,NaN,0.072970,7551,7749.459
1,1,1.0,3.0,10.0,02,2.0,1.312402,0.553140,0.547939,0.433757,0.695709,20.680199,0.047276,229630,301366.932
2,1,1.0,3.0,10.0,03,2.0,1.163921,0.684975,0.675405,0.545255,0.794220,66.359880,0.070735,527954,614496.506
3,2,1.0,7.0,50.0,04,1.0,1.062376,0.730909,0.718078,0.604134,0.820295,160.327954,0.147840,1921,2040.825


In [4]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,2.0,1.191843,0.639715,0.631254,0.506972,0.759538,52.326447,718020,855767.212
1,1,1.0,3.0,10.0,03,2.0,1.425208,0.722746,0.717922,0.578247,0.834374,59.535898,49036,69886.510


# Zonation 3

## Protection distribution

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Pirkanmaa.gpkg"

#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YourPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 767056
treestand 1498783
treestandsummary 1526105
specialfeature 1526105
number of empty ages 7551
767056
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                 2
fertilityclass           2
soiltype                 2
drainagestate          149
ditchingyear        717762
thinningyear        767056
developmentclass     19920
standquality        767056
maintreespecies       7005
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           31
Zonation_median         31
Zonation_min            31
Zonation_max            31
meanage               7551
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.9196020098854721
Zonation raja-arvo top 30%:lle on 0.8010012786836354
Suojellun osuus parhaasta 10 %:sta on 23.082117034130086 %
Suojellun osuus parhaasta 30 %:sta on 13.806132469327089 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,1096.012,4243.263,20.527356,1251.245,5133.266,19.598134
2.0,4953.592,34909.334,12.426564,8419.412,77788.475,9.766406
3.0,8808.887,25436.024,25.723200,16682.678,109330.185,13.238869
4.0,1833.321,7668.524,19.294369,4951.133,38748.785,11.329845
5.0,818.541,3461.549,19.124388,1949.116,11154.378,14.874781
6.0,92.001,343.965,21.102792,292.725,1135.174,20.500400
7.0,69.514,498.204,12.244459,214.810,1246.859,14.696214


## General

In [7]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df

,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,70.0,03,2.0,0.763606,0.898534,0.888629,0.778827,0.942774,61.137875,0.168431,9921,7575.737
2.0,1,1.0,20.0,03,2.0,1.030675,0.720967,0.711288,0.574408,0.823572,49.491985,0.074536,174037,179375.662
3.0,1,1.0,10.0,03,2.0,1.238783,0.642510,0.636913,0.526664,0.752470,51.528034,0.054053,362567,449141.686
4.0,1,1.0,10.0,03,1.0,1.313429,0.613408,0.607003,0.510349,0.716951,55.335098,0.056115,168120,220813.625
5.0,1,1.0,62.0,03,1.0,1.301018,0.640753,0.631551,0.529586,0.737437,58.560366,0.094915,42438,55212.614
6.0,2,3.0,62.0,03,1.0,1.656233,0.611958,0.607465,0.505633,0.726751,67.006357,0.170148,6224,10308.394
7.0,2,1.0,50.0,03,1.0,0.860423,0.730952,0.719816,0.615520,0.815848,105.232295,0.073145,3746,3223.144
8.0,2,1.0,50.0,NaN,1.0,0.523000,0.734115,0.710871,0.599994,0.847952,121.000000,0.000000,1,0.523


In [8]:
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,A0,2.0,1.026282,0.635398,0.631206,0.519173,0.744503,NaN,0.072970,7551,7749.459
1,1,1.0,3.0,10.0,02,2.0,1.312402,0.568287,0.564482,0.459848,0.696076,20.680199,0.047276,229630,301366.932
2,1,1.0,3.0,10.0,03,2.0,1.163921,0.696080,0.687702,0.571364,0.791521,66.359880,0.070735,527954,614496.506
3,2,1.0,7.0,50.0,04,1.0,1.062376,0.737826,0.727307,0.627626,0.816520,160.327954,0.147840,1921,2040.825


In [9]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,2.0,1.191843,0.651704,0.644545,0.532462,0.757563,52.326447,718020,855767.212
1,1,1.0,3.0,10.0,03,2.0,1.425208,0.739725,0.735468,0.612951,0.835548,59.535898,49036,69886.510


# Zonation 6

## Protection distribution

In [11]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Pirkanmaa.gpkg"

#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YourPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 767056
treestand 1498783
treestandsummary 1526105
specialfeature 1526105
number of empty ages 7551
767056
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                 2
fertilityclass           2
soiltype                 2
drainagestate          149
ditchingyear        717762
thinningyear        767056
developmentclass     19920
standquality        767056
maintreespecies       7005
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           31
Zonation_median         31
Zonation_min            31
Zonation_max            31
meanage               7551
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.9294879379989869
Zonation raja-arvo top 30%:lle on 0.8087985755438826
Suojellun osuus parhaasta 10 %:sta on 26.392173198824526 %
Suojellun osuus parhaasta 30 %:sta on 15.626422444542643 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,1049.264,3736.915,21.922791,1234.474,5020.590,19.735593
2.0,4858.631,30915.939,13.581242,8404.773,75520.585,10.014581
3.0,9563.326,25273.448,27.451813,18359.391,107796.666,14.552921
4.0,2381.071,7927.358,23.098292,6284.984,38179.912,14.134710
5.0,771.707,2488.160,23.672960,2325.823,9539.048,19.602598
6.0,103.384,396.301,20.689835,409.103,1174.044,25.841125
7.0,101.360,604.026,14.369437,261.123,1337.331,16.335972


## General

In [12]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df

,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,70.0,03,2.0,0.763606,0.894285,0.888979,0.818565,0.929674,61.137875,0.168431,9921,7575.737
2.0,1,1.0,20.0,03,2.0,1.030675,0.722290,0.715332,0.612553,0.800040,49.491985,0.074536,174037,179375.662
3.0,1,1.0,10.0,03,2.0,1.238783,0.652002,0.648179,0.563447,0.733089,51.528034,0.054053,362567,449141.686
4.0,1,1.0,10.0,03,1.0,1.313429,0.622367,0.618101,0.543637,0.698062,55.335098,0.056115,168120,220813.625
5.0,1,1.0,62.0,03,1.0,1.301018,0.633879,0.628280,0.551194,0.707004,58.560366,0.094915,42438,55212.614
6.0,2,3.0,62.0,03,1.0,1.656233,0.637855,0.636224,0.556274,0.722760,67.006357,0.170148,6224,10308.394
7.0,2,1.0,50.0,03,1.0,0.860423,0.753429,0.746260,0.670933,0.813791,105.232295,0.073145,3746,3223.144
8.0,2,1.0,50.0,NaN,1.0,0.523000,0.746627,0.734349,0.697933,0.798774,121.000000,0.000000,1,0.523


In [13]:
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,A0,2.0,1.026282,0.660311,0.658251,0.572110,0.741438,NaN,0.072970,7551,7749.459
1,1,1.0,3.0,10.0,02,2.0,1.312402,0.584939,0.583190,0.501168,0.681319,20.680199,0.047276,229630,301366.932
2,1,1.0,3.0,10.0,03,2.0,1.163921,0.697939,0.691853,0.604237,0.768066,66.359880,0.070735,527954,614496.506
3,2,1.0,7.0,50.0,04,1.0,1.062376,0.788084,0.782223,0.712260,0.838952,160.327954,0.147840,1921,2040.825


In [14]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,2.0,1.191843,0.657049,0.652197,0.566392,0.735877,52.326447,718020,855767.212
1,1,1.0,3.0,10.0,03,2.0,1.425208,0.765259,0.762041,0.675014,0.831853,59.535898,49036,69886.510
